<a href="https://colab.research.google.com/github/MariyaRafalovich/Adapter/blob/master/final_of_TelegramBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot

In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackQueryHandler, ConversationHandler, CallbackContext
import re
import nest_asyncio
import asyncio

# Токен бота, определён в начале скрипта
BOT_TOKEN = "7829871831:AAFpaORedguSYYvFx8avjrLa-u_YDZuB2J0"  # Замените на реальный токен вашего бота

# Определение состояний для разговора
EVENT_SELECTION, EVENT_DETAIL, EMAIL_INPUT = range(3)

# Список мероприятий и их описания
EVENTS = {
    "Exhibition of Salvador Dali": "A large collection of Salvador Dali's works.",
    "Kesha the Parrot: A Play": "A fun, interactive play starring Kesha the parrot.",
    "CSKA vs Parma: Basketball Match": "A thrilling basketball match between CSKA and Parma.",
    "Cheburashka Musical": "A colorful musical featuring the famous character Cheburashka."
}

# Функция для начала работы с ботом и отображения списка мероприятий
async def start(update: Update, context: CallbackContext) -> int:
    keyboard = [
        [InlineKeyboardButton(event, callback_data=event)] for event in EVENTS
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(
        "Welcome! Please choose an event to sign up for:",  # Приветственное сообщение
        reply_markup=reply_markup
    )
    return EVENT_SELECTION

# Функция для отображения детального описания мероприятия
async def event_detail(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    event_name = query.data
    description = EVENTS[event_name]

    # Отображаем описание мероприятия и кнопку "Записаться"
    keyboard = [
        [InlineKeyboardButton("Sign Up", callback_data=f"signup_{event_name}")],
        [InlineKeyboardButton("Back to Event List", callback_data="back_to_event_list")]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.answer()
    await query.edit_message_text(
        text=f"Event: {event_name}\n\n{description}",
        reply_markup=reply_markup
    )
    return EVENT_DETAIL

# Функция для обработки записи на мероприятие
async def sign_up(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    event_name = query.data.replace("signup_", "")

    # Сохраняем выбранное мероприятие для пользователя
    context.user_data['event'] = event_name

    await query.answer()
    await query.edit_message_text(text="Please provide your email address to complete your sign-up:")  # Запрос на ввод e-mail
    return EMAIL_INPUT

# Функция для обработки ввода e-mail и подтверждения записи
async def handle_email(update: Update, context: CallbackContext) -> int:
    email = update.message.text

    # Проверка корректности e-mail
    if re.match(r"[^@]+@[^@]+\.[^@]+", email):
        context.user_data['email'] = email
        event_name = context.user_data['event']

        # Подтверждаем запись и показываем кнопку "Добавить в календарь"
        await update.message.reply_text(
            f"Thank you for signing up for the event: {event_name}.\n\n"
            "You can now add this event to your calendar."  # Сообщение после успешной регистрации
        )

        keyboard = [
            [InlineKeyboardButton("Add to Calendar", callback_data="add_to_calendar")],
            [InlineKeyboardButton("Back to Event List", callback_data="back_to_event_list")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await update.message.reply_text("What would you like to do next?", reply_markup=reply_markup)  # Запрос на дальнейшие действия

        return ConversationHandler.END
    else:
        await update.message.reply_text("Please enter a valid email address.")  # Сообщение, если e-mail некорректен
        return EMAIL_INPUT  # Запрос повторно, если e-mail невалиден

# Функция для добавления мероприятия в календарь
async def add_to_calendar(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    await query.answer()
    await query.edit_message_text(text="Event successfully added to your calendar!")  # Подтверждение добавления в календарь
    return ConversationHandler.END

# Функция для возврата в список мероприятий
async def back_to_event_list(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    await query.answer()

    # Показать список мероприятий снова
    keyboard = [
        [InlineKeyboardButton(event, callback_data=event)] for event in EVENTS
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.edit_message_text(
        text="Please choose an event to sign up for:",  # Показать список мероприятий
        reply_markup=reply_markup
    )
    return EVENT_SELECTION

# Основная функция для запуска бота
async def main():
    # Инициализируем приложение с токеном бота
    application = Application.builder().token(BOT_TOKEN).build()

    # Создаем обработчик разговора
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],  # Команда /start для начала
        states={
            EVENT_SELECTION: [CallbackQueryHandler(event_detail)],  # Обработка выбора мероприятия
            EVENT_DETAIL: [
                CallbackQueryHandler(sign_up, pattern="^signup_"),  # Обработка действия "Записаться"
                CallbackQueryHandler(back_to_event_list, pattern="^back_to_event_list$")  # Обработка действия "Назад"
            ],
            EMAIL_INPUT: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_email)],  # Обработка ввода e-mail
        },
        fallbacks=[CallbackQueryHandler(add_to_calendar, pattern="^add_to_calendar$")]  # Обработка действия "Добавить в календарь"
    )

    # Добавляем обработчик в приложение
    application.add_handler(conv_handler)

    # Запуск бота с polling
    await application.run_polling()

if __name__ == '__main__':

    asyncio.run(main())  # Запуск бота


<ipython-input-3-d987e1c65aba>:123: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv_handler = ConversationHandler(


RuntimeError: Cannot close a running event loop